# Trabalho TP3 Ex 1
## Grupo 27
### LCC 2024/2025
#### Gonçalo Gonçalves Barroso A102931
#### Rafaela Antunes Pereira A102527
#### Ricardo Eusebio Cerqueira A102878

In [13]:
%pip install pysmt
%pip install z3-solver
%pip install pysmt-install --msat


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.

Usage:   
  /bin/python3 -m pip install [options] <requirement specifier> [package-index-options] ...
  /bin/python3 -m pip install [options] -r <requirements file> [package-index-options] ...
  /bin/python3 -m pip install [options] [-e] <vcs project url> ...
  /bin/python3 -m pip install [options] [-e] <local project path> ...
  /bin/python3 -m pip install [options] <archive url/path> ...

no such option: --msat
Note: you may need to restart the kernel to use updated packages.


In [14]:
from pysmt.typing import INT
from pysmt.shortcuts import *
from pysmt.shortcuts import BV, BVType, GE, Solver, Symbol



import itertools

O algoritmo estendido de Euclides (EXA) aceita dois inteiros constantes  $\,a,b>0\,$  e devolve inteiros $r,s,t\,$ tais que  $\,a*s + b*t = r\,$  e  $\,r = \gcd(a,b)\,$. 

Para além das variáveis $\,r,s,t\,$ o código requer 3 variáveis adicionais $\,r',s',t'\,$ que representam os valores de $\,r,s,t\,$ no “próximo estado”.

INPUT  a, b

assume  a > 0 and b > 0

r, r', s, s', t, t' = a, b, 1, 0, 0, 1

while r' != 0

  q = r div r'

  r, r', s, s', t, t' = r', r − q × r', s', s − q × s', t', t − q × t' 
  
OUTPUT r, s, t

#### Pretende-se

#### a. 
    Construa um SFOTS usando BitVector’s de tamanho $n$ que descreva o comportamento deste programa.  Considere estado de erro quando $\,r=0\,$ ou alguma das variáveis atinge o “overflow”.

#### b.
    Prove, usando a metodologia dos invariantes e interpolantes, que o modelo nunca atinge o estado de erro.



#### a.  Construa um SFOTS usando BitVector’s de tamanho $n$ que descreva o comportamento deste programa.  Considere estado de erro quando $\,r=0\,$ ou alguma das variáveis atinge o “overflow”.

Para representar o programa como um SFOTS (Sistema Finito de Estado Simples), consideramos o conjunto de variavies de estado X = ['pc', 'r', 'r_prime', 's', 's_prime', 't', 't_prime', 'q']

Para criar as variaveis do modelo, é definida uma função chamada gState. Esta função recebe como argumentos uma lista com os nomes das variaveis de estado, uma etiqueta, um indice inteiro e o número de bits. Com isso, ela gera a i-ésima versão das varáveis de estado, rotulada pela etiqueta fornecida. cada variável lógica segue um formato padrão, onde o nome base da variável de estado é seguido por um delimitador, criando uma nomenclatura uniforme para as diferentes instâncias.

In [15]:
def gState(vars, prefix, i, nBits):
    state = {}
    for v in vars:
        state[v] = Symbol(f'{v}_{prefix}_{i}', BVType(nBits))
    return state


Para especificar o estado inicial, é criada uma função chamada init. Esta função recebe como parâmetros um estado potencial do programa (representado por um dicionário de variáveis), dois inteiros $a$ e $b$, e o número de bits. Ela retorna um predicado do pySMT que verifica se as condições $$ r=a, r_prime=b, s=1, s_prime = 0, t = 0, t_prime = 1, r≥1, r_prime≥1 $$ são todas satisfatórias, ou seja, se o estado fornecido pode ser considerado um estado inicial válido do programa. Também verifica se o valor $a$ e $b$ está dentro dos números possíves para poder avançar no algoritmo

In [16]:
def init(state, a, b, nBits):
    
    min_value = 0
    max_value = (1 << (nBits - 1)) - 1
    
    if a < min_value or a > max_value or b < min_value or b > max_value:
        raise ValueError(f"O valor de a deve estar entre {min_value} e {max_value}")
    
    return And(
        Equals(state['pc'], BV(0, nBits)),
        Equals(state['r'], BV(a, nBits)),
        Equals(state['r_prime'], BV(b, nBits)),
        Equals(state['s'], BV(1, nBits)),
        Equals(state['s_prime'], BV(0, nBits)),
        Equals(state['t'], BV(0, nBits)),
        Equals(state['t_prime'], BV(1, nBits)),
        Equals(state['q'], BV(0, nBits))
    )


A função *error* é responsável por verificar se um determinado estado do programa contém valores que excedem os limites representáveis com um número fixo de bits, *nBits*. 

Esta retorna um resultado que indica a ocorrência de um erro se qualquer uma das variáveis estiver fora do intervalo permitido, ajudando assim a garantir a integridade e a segurança do programa ao prevenir o uso de valores inválidos ou excessivos.

In [17]:
def error(state, nBits):
    max_value = (1 << nBits) - 1
    return Or(
        BVUGT(state['r'], BV(max_value, nBits)),
        BVUGT(state['r_prime'], BV(max_value, nBits)),
        BVUGT(state['s'], BV(max_value, nBits)),
        BVUGT(state['s_prime'], BV(max_value, nBits)),
        BVUGT(state['t'], BV(max_value, nBits)),
        BVUGT(state['t_prime'], BV(max_value, nBits)),
        BVUGT(state['q'], BV(max_value, nBits)),
        Equals(state['r'], BV(0, nBits))  # r == 0 é um estado inválido
    )

A função *trans* é responsável por estabelecer as relações de transição entre dois estados do programa, retornando um predicado do pySMT que determina se a transição do primeiro estado para o segundo é válida.

Dentro da função, diversas condições são avaliadas para descrever como as variáveis de estado mudam entre os dois estados.


In [18]:
def trans(curr, prox, nBits):
    
    t01 = And(
        Equals(curr['pc'], BV(0, nBits)),
        NotEquals(curr['r_prime'], BV(0, nBits)),
        Equals(prox['pc'], BV(1, nBits)),
        Equals(prox['r'], curr['r']),
        Equals(prox['r_prime'], curr['r_prime']),
        Equals(prox['s'], curr['s']),
        Equals(prox['s_prime'], curr['s_prime']),
        Equals(prox['t'], curr['t']),
        Equals(prox['t_prime'], curr['t_prime']),
        Equals(prox['q'], curr['q'])
    )
    
    t12 = And(
        Equals(curr['pc'], BV(1, nBits)),
        Equals(prox['pc'], BV(2, nBits)),
        Equals(prox['q'], BVUDiv(curr['r'], curr['r_prime'])),
        Equals(prox['r'], curr['r']),
        Equals(prox['s'], curr['s']),
        Equals(prox['t'], curr['t']),
        Equals(prox['r_prime'], curr['r_prime']),
        Equals(prox['s_prime'], curr['s_prime']),
        Equals(prox['t_prime'], curr['t_prime'])
    )
    
    t20 = And(
        Equals(curr['pc'], BV(2, nBits)),
        Equals(prox['pc'], BV(0, nBits)),
        Equals(prox['r'], curr['r_prime']),
        Equals(prox['s'], curr['s_prime']),
        Equals(prox['t'], curr['t_prime']),
        Equals(prox['r_prime'], BVSub(curr['r'], BVMul(curr['q'], curr['r_prime']))),
        Equals(prox['s_prime'], BVSub(curr['s'], BVMul(curr['q'], curr['s_prime']))),
        Equals(prox['t_prime'], BVSub(curr['t'], BVMul(curr['q'], curr['t_prime']))),
        Equals(prox['q'], curr['q'])
    )
    
    t03 = And(
        Equals(curr['pc'], BV(0, nBits)),
        Equals(curr['r_prime'], BV(0, nBits)),
        Equals(prox['pc'], BV(3, nBits)),
        Equals(prox['r'], curr['r']),
        Equals(prox['s'], curr['s']),
        Equals(prox['t'], curr['t']),
        Equals(prox['r_prime'], curr['r_prime']),
        Equals(prox['s_prime'], curr['s_prime']),
        Equals(prox['t_prime'], curr['t_prime']),
        Equals(prox['q'], curr['q'])
    )
    
    return Or(t01, t12, t20, t03)

Usando a função *genTrace*, geramos um possível estado de execução com n transições

In [19]:

def genTrace(vars, init, trans, error, n, a, b, nBits):

    
    s = Solver()
    X = [gState(vars, 'X', i, nBits) for i in range(n+1)]
    
    try:
        I = init(X[0], a, b, nBits)
    except Exception as e:
        print(f"Erro na inicialização: {e}")
        return

    for i in range(n):

        transition = trans(X[i], X[i+1], nBits)
        error_condition = error(X[i+1], nBits)
        
        if s.solve([I, transition, Not(error_condition)]):
            print(f"Estado {i+1} válido:")
            for v in X[i+1]:
                value = s.get_value(X[i+1][v])
                raw_value = value.constant_value()
                signed_value = raw_value if raw_value < (1 << (nBits - 1)) else raw_value - (1 << nBits)
                print(f"{v}: {signed_value}")
            I = And(I, transition)

        else:
            print(f"Erro encontrado após a iteração {i+1}.")
            break

### Resultados

In [20]:
#Exemplo 1 -> É satisfazivel
vars = ['pc', 'r', 'r_prime', 's', 's_prime', 't', 't_prime', 'q']
n = 7
nbits = 8
a = 3
b = 2
genTrace(vars, init, trans, error, n, a, b, nbits)


Estado 1 válido:
pc: 1
r: 3
r_prime: 2
s: 1
s_prime: 0
t: 0
t_prime: 1
q: 0
Estado 2 válido:
pc: 2
r: 3
r_prime: 2
s: 1
s_prime: 0
t: 0
t_prime: 1
q: 1
Estado 3 válido:
pc: 0
r: 2
r_prime: 1
s: 0
s_prime: 1
t: 1
t_prime: -1
q: 1
Estado 4 válido:
pc: 1
r: 2
r_prime: 1
s: 0
s_prime: 1
t: 1
t_prime: -1
q: 1
Estado 5 válido:
pc: 2
r: 2
r_prime: 1
s: 0
s_prime: 1
t: 1
t_prime: -1
q: 2
Estado 6 válido:
pc: 0
r: 1
r_prime: 0
s: 1
s_prime: -2
t: -1
t_prime: 3
q: 2
Estado 7 válido:
pc: 3
r: 1
r_prime: 0
s: 1
s_prime: -2
t: -1
t_prime: 3
q: 2


In [21]:
#Exemplo 2 -> Não é satisfeita a condição
vars = ['pc', 'r', 'r_prime', 's', 's_prime', 't', 't_prime', 'q']
n = 10
nbits = 8
a = 17
b = 25
genTrace(vars, init, trans, error, n, a, b, nbits)


Estado 1 válido:
pc: 1
r: 17
r_prime: 25
s: 1
s_prime: 0
t: 0
t_prime: 1
q: 0
Estado 2 válido:
pc: 2
r: 17
r_prime: 25
s: 1
s_prime: 0
t: 0
t_prime: 1
q: 0
Estado 3 válido:
pc: 0
r: 25
r_prime: 17
s: 0
s_prime: 1
t: 1
t_prime: 0
q: 0
Estado 4 válido:
pc: 1
r: 25
r_prime: 17
s: 0
s_prime: 1
t: 1
t_prime: 0
q: 0
Estado 5 válido:
pc: 2
r: 25
r_prime: 17
s: 0
s_prime: 1
t: 1
t_prime: 0
q: 1
Estado 6 válido:
pc: 0
r: 17
r_prime: 8
s: 1
s_prime: -1
t: 0
t_prime: 1
q: 1
Estado 7 válido:
pc: 1
r: 17
r_prime: 8
s: 1
s_prime: -1
t: 0
t_prime: 1
q: 1
Estado 8 válido:
pc: 2
r: 17
r_prime: 8
s: 1
s_prime: -1
t: 0
t_prime: 1
q: 2
Estado 9 válido:
pc: 0
r: 8
r_prime: 1
s: -1
s_prime: 3
t: 1
t_prime: -2
q: 2
Estado 10 válido:
pc: 1
r: 8
r_prime: 1
s: -1
s_prime: 3
t: 1
t_prime: -2
q: 2


In [22]:
#Exemplo 3 -> Erro na inicialização
vars = ['pc', 'r', 'r_prime', 's', 's_prime', 't', 't_prime', 'q']
n = 10
nbits = 8
a = 128
b = 6
genTrace(vars, init, trans, error, n, a, b, nbits)

Erro na inicialização: O valor de a deve estar entre 0 e 127


#### b. Prove, usando a metodologia dos invariantes e interpolantes, que o modelo nunca atinge o estado de erro.


In [23]:
def initB(state, nbits):
    # Definição dos estados iniciais
    a = BVSGT(state['a'], BV(0, nbits))
    b = BVSGT(state['b'], BV(0, nbits))
    r = Equals(state['r'], state['a'])
    r_prime = Equals(state['r_prime'], state['b'])
    s = Equals(state['s'], BV(1, nbits))
    s_prime = Equals(state['s_prime'], BV(0, nbits))
    t = Equals(state['t'], BV(0, nbits))
    t_prime = Equals(state['t_prime'], BV(1, nbits))
    pc = Equals(state['pc'], BV(0, nbits))
    q = Equals(state['q'], BV(0, nbits))

    # Combinação de todas as condições
    return And(a,b,r, r_prime, s, s_prime, t, t_prime, pc, q)


Agora, usando *funções* que nos foram dadas nas fichas, conseguimos, usando a metodologia dos invariantes e interpolantes, provar que o modelo não atinge o estado de erro.


In [24]:
def invert(trans):
    return lambda curr, prox, nbits: trans(prox, curr, nbits)

def baseName(s):
    return ''.join(list(itertools.takewhile(lambda x: x!='!', s)))

def rename(form, state):
    vs = get_free_variables(form)
    pairs = []
    for x in vs:
        key = baseName(x.symbol_name())
        if key in state:
            pairs.append((x, state[key]))
    return form.substitute(dict(pairs))

def same(state1,state2):
    return And([Equals(state1[x],state2[x]) for x in state1])

def model_checking(vars,initB,trans,error,N,M,nbits):
    with Solver(name="msat") as solver:

        # Criar todos os estados que poderão vir a ser necessários.
        X = [gState(vars,'X',i,nbits) for i in range(N+1)]
        Y = [gState(vars,'Y',i,nbits) for i in range(M+1)]
        transt = invert(trans)

        # Estabelecer a ordem pela qual os pares (n,m) vão surgir. Por exemplo:
        order = sorted([(a,b) for a in range(1,N+1) for b in range(1,M+1)],key=lambda tup:tup[0]+tup[1])

        # Step 1 implícito na ordem de 'order' e nas definições de Rn, Um.
        for (n,m) in order:
            # Step 2.
                
            I = initB(X[0],nbits)
            Tn = And([trans(X[i], X[i+1], nbits) for i in range(n)])
            Rn = And(I, Tn)

            E = error(Y[0], nbits)
            Bm = And([transt(Y[i], Y[i+1], nbits) for i in range(m)])
            Um = And(E, Bm)

            Vnm = And(Rn, same(X[n], Y[m]), Um)
            if solver.solve([Vnm]):
                print("> O sistema é inseguro.")
                return
            else:
                # Step 3.
                A = And(Rn, same(X[n], Y[m]))
                B = Um
                C = binary_interpolant(A, B)

                # Salvaguardar cálculo bem-sucedido do interpolante.
                if C is None:
                    print("> O interpolante é None.")
                    break

                # Step 4.
                C0 = rename(C, X[0])
                T = trans(X[0], X[1], nbits)
                C1 = rename(C, X[1])

                if not solver.solve([C0, T, Not(C1)]):
                    # C é invariante de T.
                    print("> O sistema é seguro.")
                    return
                else:
                    # Step 5.1.
                    S = rename(C, X[n])
                    while True:
                        # Step 5.2.
                        T = trans(X[n], Y[m], nbits)
                        A = And(S, T)
                        if solver.solve([A, Um]):
                            print("> Não foi encontrado majorante.")
                            break
                        else:
                            # Step 5.3.
                            C = binary_interpolant(A, Um)
                            Cn = rename(C, X[n])
                            if not solver.solve([Cn, Not(S)]):
                                # Step 5.4.
                                # C(Xn) -> S é tautologia.
                                print("> O sistema é seguro.")
                                return
                            else:
                                # Step 5.5.
                                # C(Xn) -> S não é tautologia.
                                S = Or(S, Cn)

    print("> Não foi provada a segurança ou insegurança do sistema.")

varsB = ['pc', 'r', 'r_prime', 's', 's_prime', 't', 't_prime', 'q', 'a', 'b']
model_checking(varsB, initB, trans, error, 50, 50, 8)

> O sistema é seguro.
